# Test translation

In [1]:
import ctypes
import cv2
import numpy as np
import time

In [2]:
ekmi = ctypes.cdll.LoadLibrary('./ekmi.so')
_get_ekmi = ekmi.get_ekmi
_get_ekmi.argtypes = [
    ctypes.c_int, 
    np.ctypeslib.ndpointer(dtype=np.uintp, flags="C", ndim=1), 
    ctypes.c_int,
    ctypes.c_double
]
_get_ekmi.restype = ctypes.POINTER(ctypes.POINTER(ctypes.c_double))


def get_ekmi(order, img, p):
    
    imgpp = (img.__array_interface__['data'][0] + np.arange(img.shape[0])* img.strides[0]).astype(np.uintp) 
    _res = _get_ekmi(order, imgpp, img.shape[0], p)

    # copy array
    res = np.zeros((order, order))
    for i in range(order):
        for j in range(order):
            res[i, j] = _res[i][j]
    ekmi.free_ekmi_rst() # free memory
    return res

In [23]:
def ekmi_from_fname(fname):
    p = 0.5
    N = 16
    tik = time.time()
    order = 6

    img = (cv2.imread(fname)[:, :, 0] == 255).astype('int32')
    retval = get_ekmi(order, img, p)
    tok = time.time()
    print(tok - tik)
    return retval

k1 = ekmi_from_fname("/Users/kx/desktop/1_small.png")
k2 = ekmi_from_fname("/Users/kx/desktop/1_small_t.png")
print(k1)
print(k2)
sum(sum((k2 - k1) ** 2))

0.003990888595581055
0.004524946212768555
[[ 6.76287898e-02  3.96066258e-02  2.17033692e-02  1.13113460e-02
   6.16732109e-03  4.35971204e-03]
 [ 3.08754137e-02  1.80185271e-02  9.78166885e-03  4.97825326e-03
   2.57842594e-03  1.71231016e-03]
 [ 1.08157990e-02  6.34668777e-03  3.46160123e-03  1.75683363e-03
   8.82037852e-04  5.41326979e-04]
 [ 1.55718235e-03  1.00129988e-03  6.20505621e-04  3.72881403e-04
   2.22102369e-04  1.37565227e-04]
 [-1.37842190e-03 -6.93965503e-04 -2.72222353e-04 -4.41075874e-05
   5.03897232e-05  6.20406283e-05]
 [-1.21979183e-03 -6.28242088e-04 -2.55107783e-04 -4.42184482e-05
   5.32040774e-05  7.84118394e-05]]
[[ 6.76287898e-02  3.96066258e-02  2.17033692e-02  1.13113460e-02
   6.16732109e-03  4.35971204e-03]
 [ 3.08754137e-02  1.80185271e-02  9.78166885e-03  4.97825326e-03
   2.57842594e-03  1.71231016e-03]
 [ 1.08157990e-02  6.34668777e-03  3.46160123e-03  1.75683363e-03
   8.82037852e-04  5.41326979e-04]
 [ 1.55718235e-03  1.00129988e-03  6.20505621e-0

7.664306597986357e-25

___

# Test recognition

In [5]:
import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsClassifier
from ikmi import *

In [6]:
order = 5
p = 0.5

In [7]:
data_dir = '/Users/kx/Docs/github/git_cccr_sl/chinese-calligraphy-recognition-sl/data/shufadict/clean'
chars = [x for x in os.listdir(data_dir) if x[0] != '.']
N = 32
nclass = 10
sample_per_class = 10

In [8]:
class_lookup = [None] * nclass
data = np.zeros((nclass * sample_per_class, N, N), dtype='int32')
label = np.zeros((nclass * sample_per_class), dtype='int')
feature_ekmi = np.zeros((nclass * sample_per_class, order, order))
feature_ikmi = np.zeros((nclass * sample_per_class, order, order))

In [9]:
for i, char in enumerate(chars[:nclass]):
    class_lookup[i] = char
    fnames = glob.glob('{}/{}/*.png'.format(data_dir, char))[:sample_per_class]
    for j, fname in enumerate(fnames):
        img = cv2.imread(fname)
        img = cv2.resize(img, (N, N))
        img = (img[:, :, 0] > 0).astype('int32')
        data[i * nclass + j, :, :] = img
        label[i * nclass + j] = i

In [10]:
for i in tqdm(range(nclass * sample_per_class)):
    img = data[i, :, :]
    feature_ekmi[i, :, :] = get_ekmi(order, img, p)
    feature_ikmi[i, :, :] = wkrchkmoment_single(img)[0][:order, :order]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
# todo: choose feature
feature = feature_ekmi

In [12]:
sum(sum((feature[1] - feature[12]) ** 2))

0.004018112291240717

In [22]:
label[1], label[30]

(0, 3)

In [20]:
(feature[1,:,:] * 100).round(2)

array([[3.61, 1.72, 0.76, 0.3 , 0.11],
       [1.59, 0.77, 0.34, 0.14, 0.05],
       [0.62, 0.31, 0.14, 0.06, 0.02],
       [0.2 , 0.1 , 0.05, 0.02, 0.01],
       [0.05, 0.03, 0.01, 0.01, 0.  ]])

In [21]:
(feature[30,:,:] * 100).round(2)

array([[ 2.28,  0.53,  0.09,  0.  , -0.  ],
       [ 1.58,  0.36,  0.06,  0.  , -0.  ],
       [ 1.07,  0.23,  0.03,  0.  , -0.  ],
       [ 0.7 ,  0.15,  0.02,  0.  , -0.  ],
       [ 0.45,  0.09,  0.01, -0.  , -0.  ]])

In [13]:
test_idxs = np.concatenate([[i * nclass + j for i in range(nclass)] for j in range(2)])
test_idxs.sort()
train_idxs = [x for x in range(feature.shape[0]) if x not in test_idxs]

test = feature[test_idxs]
test_label = label[test_idxs]
test_feature = feature[test_idxs]

train = feature[train_idxs]
train_label = label[train_idxs]
train_feature = feature[train_idxs]

In [14]:
X = train.reshape((train.shape[0], train.shape[1] * train.shape[2]))
y = test.reshape((test.shape[0], test.shape[1] * test.shape[2]))

In [15]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, train_label)
sum(knn.predict(y) == test_label) / len(test_label)

0.25

In [19]:
# dist = np.zeros((train.shape[0], train.shape[0]))
# for i in range(train.shape[0]):
#     for j in range(train.shape[0]):
#         dist[i, j] = sum(sum((feature[i, :, :] - feature[j, :, :]) ** 2))
        
# np.where((dist > 0) & (dist < 1))